# Advanced Quantum Algorithms with Qiskit

This notebook implements key quantum algorithms using Qiskit with optimizations from recent research. We'll cover Shor's algorithm for factorization, Grover's search algorithm, Deutsch-Jozsa algorithm, and Simon's algorithm with performance optimizations.

## Import Required Libraries

In [ ]:
# Import Qiskit and related libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Core Qiskit imports
import qiskit
from qiskit import QuantumCircuit, Aer, execute, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.algorithms import Shor, Grover, AmplificationProblem
from qiskit.quantum_info import Statevector

# Qiskit tools for optimization
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.backends import QasmSimulator
from qiskit.providers.aer import AerError
from qiskit.utils import QuantumInstance

# Advanced modules for optimization
from qiskit.circuit.library import QFT
from qiskit.circuit.library.arithmetic import ModularMultiplier, ModularExponentiation

print(f"Qiskit version: {qiskit.__version__}")

ModuleNotFoundError: No module named 'numpy'

Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:42733\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.']
Bad pipe message: %s [b'0.0 Safari/537.36\r\nAccept-Encoding: gzip, defla']
Bad pipe message: %s [b', br, zstd\r\nAccept-Language: en-US,en;q=0.9\r\nCache-Control: max-age=0\r\nReferer: https://github.com/\r\nX-Request-ID: ', b'3d61d588292ba4062a78ac9d37b9e1\r\nX-Real-IP: 167.89.2', b'.148\r\nX-Forwarded-Port: 443\r\nX-Forwarded-Scheme']
Bad pipe message: %s [b'https\r\nX-Original-URI: /\r\nX-Scheme: https\r\nsec-fetch-site']
Bad pipe message: %s [b'cross-site\r\nsec-fetch-mode: navigate\r\nsec-fetch-dest: doc']
Bad pipe message: %s [b'ent\r\nsec-ch-ua: "Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-pl', b'form: "Windows"\r\npriority: u=0, i\r\nX-Original-Proto: https\r\nX-Forwarded-Proto

## Shor's Algorithm (Optimized Factorization)

Shor's algorithm provides an exponential speedup for integer factorization compared to classical algorithms. Here we implement it with:
- Approximate QFT for circuit depth reduction
- Optimized modular exponentiation gates
- Post-processing optimizations

In [ ]:
def run_optimized_shor(N, a=None):
    """
    Run Shor's algorithm with optimizations for factoring N
    
    Args:
        N (int): Number to factor, must be odd and > 1
        a (int): Optional co-prime to N for the algorithm
        
    Returns:
        tuple: Factors of N if successful, or None
    """
    if N % 2 == 0:
        print(f"N={N} is even, returning trivial factorization")
        return 2, N//2
    
    # Create quantum instance with optimization level
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(
        backend=backend,
        shots=1024,
        optimization_level=3,  # Maximum optimization
        memory=True
    )
    
    # If a is not provided, choose a random coprime
    if a is None:
        import math
        import random
        a = random.randint(2, N-1)
        while math.gcd(a, N) != 1:
            a = random.randint(2, N-1)
    
    print(f"Running Shor's algorithm to factor N={N} using a={a}")
    
    # Create Shor's algorithm instance with optimization flag
    shor = Shor(
        quantum_instance=quantum_instance,
        use_approximation=True  # Use approximate QFT for depth reduction
    )
    
    try:
        result = shor.factor(N, a=a)
        print(f"Factors: {result.factors}")
        return result.factors[0]
    except Exception as e:
        print(f"Error in Shor's algorithm: {e}")
        return None

In [ ]:
# Example: Factor 15 using optimized Shor's algorithm
factors = run_optimized_shor(15, a=7)
print(f"Factorization result: {factors}")

## Grover's Algorithm (Amplitude Amplification)

Grover's algorithm provides quadratic speedup for unstructured search problems. Below we implement Grover's algorithm with:
- Custom oracle construction
- Amplitude amplification
- Optimized diffusion operators

In [ ]:
def create_grover_oracle(n_qubits, target_state):
    """
    Create a custom oracle for Grover's algorithm that marks the target state
    
    Args:
        n_qubits (int): Number of qubits in the circuit
        target_state (str): Binary string representing the target state
        
    Returns:
        QuantumCircuit: Oracle circuit that applies phase flip to target state
    """
    # Create quantum circuit with n qubits
    oracle_circuit = QuantumCircuit(n_qubits)
    
    # Convert target state string to list of bit values
    target_bits = [int(bit) for bit in target_state]
    
    # Apply X gates to qubits where target bit is 0
    for qubit, bit in enumerate(target_bits):
        if bit == 0:
            oracle_circuit.x(qubit)
            
    # Apply multi-controlled Z gate
    oracle_circuit.h(n_qubits-1)
    oracle_circuit.mcx(list(range(n_qubits-1)), n_qubits-1)
    oracle_circuit.h(n_qubits-1)
    
    # Apply X gates again to qubits where target bit is 0
    for qubit, bit in enumerate(target_bits):
        if bit == 0:
            oracle_circuit.x(qubit)
    
    oracle_circuit.name = "Oracle"
    return oracle_circuit

def run_grover_search(n_qubits, target_state):
    """
    Run Grover's algorithm to find the target state
    
    Args:
        n_qubits (int): Number of qubits
        target_state (str): Binary string representing target state
        
    Returns:
        dict: Results from the Grover search
    """
    # Create oracle
    oracle = create_grover_oracle(n_qubits, target_state)
    
    # Calculate optimal number of iterations
    iterations = int(np.pi/4 * np.sqrt(2**n_qubits))
    print(f"Optimal number of Grover iterations: {iterations}")
    
    # Set up the Grover search problem
    problem = AmplificationProblem(
        oracle=oracle,
        state_preparation=None,  # Default uniform superposition
        is_good_state=lambda x: x == int(target_state, 2)
    )
    
    # Create and run Grover's algorithm
    grover = Grover(quantum_instance=Aer.get_backend('qasm_simulator'))
    result = grover.amplify(problem)
    
    return result


In [ ]:
# Example: Use Grover's algorithm to find the target state "101" in a 3-qubit system
n_qubits = 3
target_state = "101"  # State we're searching for

result = run_grover_search(n_qubits, target_state)
print(f"Grover search result: {result.top_measurement}")

# Visualize the results
if hasattr(result, 'circuit_results') and result.circuit_results:
    plot_histogram(result.circuit_results[0])

## Deutsch-Jozsa Algorithm (Constant/Balanced Detection)

The Deutsch-Jozsa algorithm determines whether a function is constant or balanced with a single query. We'll implement it with a dynamic oracle constructor that supports both constant and balanced functions.

In [ ]:
def create_deutsch_jozsa_oracle(n_qubits, function_type='balanced'):
    """
    Create an oracle for the Deutsch-Jozsa algorithm
    
    Args:
        n_qubits (int): Number of qubits in the circuit
        function_type (str): Type of function ('constant' or 'balanced')
        
    Returns:
        QuantumCircuit: Oracle circuit
    """
    oracle_circuit = QuantumCircuit(n_qubits + 1)
    
    if function_type == 'constant':
        # Apply X gate to the last qubit if the function is constant and returns 1
        oracle_circuit.x(n_qubits)
    elif function_type == 'balanced':
        # Apply CNOT gates to create a balanced function
        for qubit in range(n_qubits):
            oracle_circuit.cx(qubit, n_qubits)
    
    oracle_circuit.name = "Oracle"
    return oracle_circuit

def run_deutsch_jozsa(n_qubits, function_type='balanced'):
    """
    Run the Deutsch-Jozsa algorithm
    
    Args:
        n_qubits (int): Number of qubits
        function_type (str): Type of function ('constant' or 'balanced')
        
    Returns:
        str: 'constant' or 'balanced'
    """
    # Create oracle
    oracle = create_deutsch_jozsa_oracle(n_qubits, function_type)
    
    # Create Deutsch-Jozsa circuit
    dj_circuit = QuantumCircuit(n_qubits + 1, n_qubits)
    
    # Initialize last qubit in |1>
    dj_circuit.x(n_qubits)
    dj_circuit.h(n_qubits)
    
    # Apply Hadamard gates to the first n qubits
    for qubit in range(n_qubits):
        dj_circuit.h(qubit)
    
    # Append oracle
    dj_circuit.append(oracle, range(n_qubits + 1))
    
    # Apply Hadamard gates to the first n qubits again
    for qubit in range(n_qubits):
        dj_circuit.h(qubit)
    
    # Measure the first n qubits
    dj_circuit.measure(range(n_qubits), range(n_qubits))
    
    # Execute the circuit
    backend = Aer.get_backend('qasm_simulator')
    result = execute(dj_circuit, backend, shots=1024).result()
    counts = result.get_counts()
    
    # Analyze the results
    if '0' * n_qubits in counts:
        return 'constant'
    else:
        return 'balanced'


In [ ]:
# Example: Run Deutsch-Jozsa algorithm with a balanced function
n_qubits = 3
function_type = 'balanced'

result = run_deutsch_jozsa(n_qubits, function_type)
print(f"Deutsch-Jozsa result: {result}")

## Simon's Algorithm (Periodicity Detection)

Simon's algorithm finds the period of a given function with an exponential speedup over classical algorithms. We'll implement it with:
- Efficient oracle construction
- Optimized measurement and post-processing

In [ ]:
def create_simon_oracle(n_qubits, secret_string):
    """
    Create an oracle for Simon's algorithm
    
    Args:
        n_qubits (int): Number of qubits in the circuit
        secret_string (str): Binary string representing the secret period
        
    Returns:
        QuantumCircuit: Oracle circuit
    """
    oracle_circuit = QuantumCircuit(2 * n_qubits)
    
    # Apply CNOT gates based on the secret string
    for i, bit in enumerate(secret_string):
        if bit == '1':
            oracle_circuit.cx(i, n_qubits + i)
    
    oracle_circuit.name = "Oracle"
    return oracle_circuit

def run_simon_algorithm(n_qubits, secret_string):
    """
    Run Simon's algorithm to find the secret period
    
    Args:
        n_qubits (int): Number of qubits
        secret_string (str): Binary string representing the secret period
        
    Returns:
        str: Secret period
    """
    # Create oracle
    oracle = create_simon_oracle(n_qubits, secret_string)
    
    # Create Simon's algorithm circuit
    simon_circuit = QuantumCircuit(2 * n_qubits, n_qubits)
    
    # Apply Hadamard gates to the first n qubits
    for qubit in range(n_qubits):
        simon_circuit.h(qubit)
    
    # Append oracle
    simon_circuit.append(oracle, range(2 * n_qubits))
    
    # Apply Hadamard gates to the first n qubits again
    for qubit in range(n_qubits):
        simon_circuit.h(qubit)
    
    # Measure the first n qubits
    simon_circuit.measure(range(n_qubits), range(n_qubits))
    
    # Execute the circuit
    backend = Aer.get_backend('qasm_simulator')
    result = execute(simon_circuit, backend, shots=1024).result()
    counts = result.get_counts()
    
    # Post-process the results to find the secret period
    # This is a simplified version, real implementation would involve solving linear equations
    secret_period = max(counts, key=counts.get)
    return secret_period


In [ ]:
# Example: Run Simon's algorithm with a secret period "110"
n_qubits = 3
secret_string = '110'

result = run_simon_algorithm(n_qubits, secret_string)
print(f"Simon's algorithm result: {result}")

## Quantum Optimization Module

This module integrates quantum optimization techniques to solve complex problems. We'll implement a Quantum Approximate Optimization Algorithm (QAOA) and a Variational Quantum Eigensolver (VQE) for optimization tasks.

In [ ]:
from qiskit.algorithms import QAOA, VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA, SLSQP
from qiskit.opflow import PauliSumOp, X, Y, Z, I

class QuantumOptimizer:
    def __init__(self, qubit_count=8):
        self.qubit_count = qubit_count
        self.simulator = Aer.get_backend('qasm_simulator')
        self.statevector_sim = Aer.get_backend('statevector_simulator')
        self.optimizer_type = 'COBYLA'  # Default classical optimizer
        self.ansatz_type = 'TwoLocal'  # Default ansatz
        self.shots = 1024
        self.max_iterations = 100
        self.noise_model = None  # For realistic hardware simulation
        
    def create_ansatz(self, layers=3, entanglement='full'):
        if self.ansatz_type == 'TwoLocal':
            return TwoLocal(self.qubit_count, ['ry', 'rz'], 'cz', reps=layers, entanglement=entanglement)
        else:
            raise ValueError(f"Unknown ansatz type: {self.ansatz_type}")
        
    def solve_optimization_problem(self, cost_function, initial_point=None):
        ansatz = self.create_ansatz(layers=3)
        num_parameters = ansatz.num_parameters
        if initial_point is None:
            initial_point = np.random.random(num_parameters)
        
        if self.optimizer_type == 'COBYLA':
            optimizer = COBYLA(maxiter=self.max_iterations)
        elif self.optimizer_type == 'SLSQP':
            optimizer = SLSQP(maxiter=self.max_iterations)
        else:
            raise ValueError(f"Unknown optimizer: {self.optimizer_type}")
        
        def objective_function(parameters):
            bound_circuit = ansatz.bind_parameters(parameters)
            result = execute(bound_circuit, self.simulator, shots=self.shots).result()
            counts = result.get_counts()
            cost = cost_function(counts)
            return cost
        
        result = minimize(objective_function, initial_point, method=self.optimizer_type, options={'maxiter': self.max_iterations})
        return result
        
    def solve_qaoa(self, qubo_matrix):
        hamiltonian = self._qubo_to_ising(qubo_matrix)
        p_layers = 3
        ansatz = QAOAAnsatz(hamiltonian, p_layers)
        optimizer = COBYLA(maxiter=self.max_iterations)
        qaoa = QAOA(optimizer=optimizer, quantum_instance=self.simulator)
        result = qaoa.compute_minimum_eigenvalue(hamiltonian)
        return result
        
    def _qubo_to_ising(self, qubo_matrix):
        n = qubo_matrix.shape[0]
        hamiltonian = 0
        for i in range(n):
            for j in range(i, n):
                if i == j:
                    coeff = qubo_matrix[i, i]
                    term = (I - Z) @ i
                    hamiltonian += coeff * term / 2
                else:
                    coeff = qubo_matrix[i, j]
                    term = (I - Z) @ i * (I - Z) @ j
                    hamiltonian += coeff * term / 4
        return hamiltonian
        
    def visualize_circuit(self, circuit=None):
        if circuit is None:
            circuit = self.create_ansatz()
        return circuit.draw(output='mpl')
        
    def analyze_cost_landscape(self, cost_function, param_range=(-np.pi, np.pi), resolution=20):
        simple_qc = QuantumCircuit(2)
        simple_qc.rx(0, 0)
        simple_qc.ry(0, 1)
        simple_qc.cx(0, 1)
        param_vals = np.linspace(param_range[0], param_range[1], resolution)
        cost_landscape = np.zeros((resolution, resolution))
        for i, theta1 in enumerate(param_vals):
            for j, theta2 in enumerate(param_vals):
                params = [theta1, theta2]
                bound_circuit = simple_qc.bind_parameters(params)
                result = execute(bound_circuit, self.simulator, shots=self.shots).result()
                counts = result.get_counts()
                cost = cost_function(counts)
                cost_landscape[i, j] = cost
        return param_vals, cost_landscape


## Quantum Computing Module for Complex Computations and Optimizations

This module integrates quantum computing techniques to perform complex computations and optimizations. We'll implement a Quantum Approximate Optimization Algorithm (QAOA) and a Variational Quantum Eigensolver (VQE) for optimization tasks.

In [ ]:
from qiskit.algorithms import QAOA, VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA, SLSQP
from qiskit.opflow import PauliSumOp, X, Y, Z, I

class QuantumComputingModule:
    def __init__(self, qubit_count=8):
        self.qubit_count = qubit_count
        self.simulator = Aer.get_backend('qasm_simulator')
        self.statevector_sim = Aer.get_backend('statevector_simulator')
        self.optimizer_type = 'COBYLA'  # Default classical optimizer
        self.ansatz_type = 'TwoLocal'  # Default ansatz
        self.shots = 1024
        self.max_iterations = 100
        self.noise_model = None  # For realistic hardware simulation
        
    def create_ansatz(self, layers=3, entanglement='full'):
        if self.ansatz_type == 'TwoLocal':
            return TwoLocal(self.qubit_count, ['ry', 'rz'], 'cz', reps=layers, entanglement=entanglement)
        else:
            raise ValueError(f"Unknown ansatz type: {self.ansatz_type}")
        
    def solve_optimization_problem(self, cost_function, initial_point=None):
        ansatz = self.create_ansatz(layers=3)
        num_parameters = ansatz.num_parameters
        if initial_point is None:
            initial_point = np.random.random(num_parameters)
        
        if self.optimizer_type == 'COBYLA':
            optimizer = COBYLA(maxiter=self.max_iterations)
        elif self.optimizer_type == 'SLSQP':
            optimizer = SLSQP(maxiter=self.max_iterations)
        else:
            raise ValueError(f"Unknown optimizer: {self.optimizer_type}")
        
        def objective_function(parameters):
            bound_circuit = ansatz.bind_parameters(parameters)
            result = execute(bound_circuit, self.simulator, shots=self.shots).result()
            counts = result.get_counts()
            cost = cost_function(counts)
            return cost
        
        result = minimize(objective_function, initial_point, method=self.optimizer_type, options={'maxiter': self.max_iterations})
        return result
        
    def solve_qaoa(self, qubo_matrix):
        hamiltonian = self._qubo_to_ising(qubo_matrix)
        p_layers = 3
        ansatz = QAOAAnsatz(hamiltonian, p_layers)
        optimizer = COBYLA(maxiter=self.max_iterations)
        qaoa = QAOA(optimizer=optimizer, quantum_instance=self.simulator)
        result = qaoa.compute_minimum_eigenvalue(hamiltonian)
        return result
        
    def _qubo_to_ising(self, qubo_matrix):
        n = qubo_matrix.shape[0]
        hamiltonian = 0
        for i in range(n):
            for j in range(i, n):
                if i == j:
                    coeff = qubo_matrix[i, i]
                    term = (I - Z) @ i
                    hamiltonian += coeff * term / 2
                else:
                    coeff = qubo_matrix[i, j]
                    term = (I - Z) @ i * (I - Z) @ j
                    hamiltonian += coeff * term / 4
        return hamiltonian
        
    def visualize_circuit(self, circuit=None):
        if circuit is None:
            circuit = self.create_ansatz()
        return circuit.draw(output='mpl')
        
    def analyze_cost_landscape(self, cost_function, param_range=(-np.pi, np.pi), resolution=20):
        simple_qc = QuantumCircuit(2)
        simple_qc.rx(0, 0)
        simple_qc.ry(0, 1)
        simple_qc.cx(0, 1)
        param_vals = np.linspace(param_range[0], param_range[1], resolution)
        cost_landscape = np.zeros((resolution, resolution))
        for i, theta1 in enumerate(param_vals):
            for j, theta2 in enumerate(param_vals):
                params = [theta1, theta2]
                bound_circuit = simple_qc.bind_parameters(params)
                result = execute(bound_circuit, self.simulator, shots=self.shots).result()
                counts = result.get_counts()
                cost = cost_function(counts)
                cost_landscape[i, j] = cost
        return param_vals, cost_landscape
